In [52]:
import json
import pandas as pd

In [53]:
patients_luminex_files = ['xxx1.xls',
                          'xxx2.xls']
N_ROWS_TO_SKIP = 10

DB_ID = None
ETAG = None

# Parse data from luminex to antibodies list

In [54]:
hla_antibodies_list = []
for luminex_file in patients_luminex_files:
    df = pd.read_excel(luminex_file, skiprows=N_ROWS_TO_SKIP)
    df = df.iloc[:, [15, 35]]
    df.columns = ['mfi', 'code']
    df = df.dropna()

    for _, row in df.iterrows():
        antibodies = [c.strip() for c in row.code.split(',') if '-' not in c]
        if len(antibodies) == 1:
            hla_antibodies_list.append({'raw_code': antibodies[0], 'mfi': int(row.mfi)})
        elif len(antibodies) == 2:
            raw_code = antibodies[0][:2]
            raw_code += f'[{antibodies[0].split("*")[1]},{antibodies[1].split("*")[1]}]'
            hla_antibodies_list.append({'raw_code': raw_code, 'mfi': int(row.mfi)})
        else:
            raise AssertionError('Expecting one or two raw codes specified in each row of luminex data.')

# Create json to update the patient

In [55]:
update_patient_dict = {'db_id': DB_ID, 'etag': ETAG, 'hla_antibodies': {'hla_antibodies_list': hla_antibodies_list}}
with open(f"../update_patient_{DB_ID}.json", "w") as file:
    json.dump(update_patient_dict, file)